In [4]:
# imports

from catalog import Catalog
from course import Course

from bs4 import BeautifulSoup
import requests

print('Imports Completed')

Imports Completed


In [5]:
# variable setup
catalog = Catalog()

catalog_url = 'http://catalogue.uci.edu/allcourses/engrmae/'
save_file_path = 'catalog.json'

print('Variables Initialized')

Variables Initialized


In [8]:
page = requests.get(catalog_url)
soup = BeautifulSoup(page.content, 'html.parser')

courses = soup.find_all('div', class_='courseblock')

for course in courses:
    course_block_title = course.find('p', class_='courseblocktitle')
    course_block_desc = course.find('div', class_='courseblockdesc')

    course_block_title_text = course_block_title.text.split(sep='.')
    section, code = course_block_title_text[0].split(sep='\xa0')
    title = course_block_title_text[1].strip()
    units = course_block_title_text[2].split()[0]

    course_block_desc_children = course_block_desc.findChildren()
    description = course_block_desc_children[0].text

    design_units = 0
    prerequisites = ''
    corequisites = ''

    for block in course_block_desc_children:
        text = block.text
        if 'Prerequisite' in text:
            if 'Corequisite' in text:
                text = text.split(sep='\n')
                corequisites = text[0].partition(' ')[2].strip().replace('\xa0', ' ')
                prerequisites = text[1].partition(' ')[2].strip().replace('\xa0', ' ')
            else:
                prerequisites = text.partition(' ')[2].strip().replace('\xa0', ' ')
        elif 'Corequisite' in text:
            corequisites = text.partition(' ')[2].strip().replace('\xa0', ' ')
        elif 'Design units' in text:
            design_units = text.strip('()').split()[-1]

    catalog.add(Course(section, code, title, units, description,
                          design_units, corequisites, prerequisites))

print('Catalog Filled')

Catalog Filled


In [10]:
catalog.save_to_file(save_file_path)

print('File Saved')

File Saved
